<a href="https://colab.research.google.com/github/nguyencong1227/Neural-networks-and-deep-learning/blob/main/Tranning_Cifar_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
num_epochs = 20
batch_size = 4
learning_rate = 0.005

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(0.2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout(0.2)
        self.fc1 = nn.LazyLinear(16 * 5 * 5, 120)
        self.fc2 = nn.LazyLinear(120, 84)
        self.fc3 = nn.LazyLinear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.drop1(x)
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = self.drop2(x)  
        x = x.view(-1, 16 * 5 * 5)            
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))               
        x = F.relu(self.fc2(x))               
        x = self.fc3(x)                       
        return x

In [ ]:
class ConvNet(nn.Module):
   

    def __init__(self):
        
        super(ConvNet, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

In [ ]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 1000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/20], Step [1000/12500], Loss: 1.2614
Epoch [1/20], Step [2000/12500], Loss: 1.9891
Epoch [1/20], Step [3000/12500], Loss: 1.8176
Epoch [1/20], Step [4000/12500], Loss: 1.9119
Epoch [1/20], Step [5000/12500], Loss: 1.3173
Epoch [1/20], Step [6000/12500], Loss: 1.2888
Epoch [1/20], Step [7000/12500], Loss: 2.9108
Epoch [1/20], Step [8000/12500], Loss: 0.9007
Epoch [1/20], Step [9000/12500], Loss: 0.7513
Epoch [1/20], Step [10000/12500], Loss: 0.8751
Epoch [1/20], Step [11000/12500], Loss: 0.6220
Epoch [1/20], Step [12000/12500], Loss: 1.0877
Epoch [2/20], Step [1000/12500], Loss: 0.5344
Epoch [2/20], Step [2000/12500], Loss: 1.0749
Epoch [2/20], Step [3000/12500], Loss: 0.4239
Epoch [2/20], Step [4000/12500], Loss: 1.2692
Epoch [2/20], Step [5000/12500], Loss: 0.6082
Epoch [2/20], Step [6000/12500], Loss: 0.7958
Epoch [2/20], Step [7000/12500], Loss: 0.4171
Epoch [2/20], Step [8000/12500], Loss: 0.3026
Epoch [2/20], Step [9000/12500], Loss: 0.2176
Epoch [2/20], Step [10000/12500

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')
        

NameError: ignored

```
Accuracy of the network: 44.39 %
Accuracy of plane: 27.3 %
Accuracy of car: 55.4 %
Accuracy of bird: 29.1 %
Accuracy of cat: 26.5 %
Accuracy of deer: 34.4 %
Accuracy of dog: 24.7 %
Accuracy of frog: 54.0 %
Accuracy of horse: 57.1 %
Accuracy of ship: 79.2 %
Accuracy of truck: 56.2 %
```

